In [2]:
from dataset import PBowDataset

datset = PBowDataset(dataset_type='train', hm_class=1, nclusters=10)


energy_frame.parquet exists. Loading now.
box_frame.parquet exists. Loading now.
force_frame.parquet exists. Loading now.
type_frame.parquet exists. Loading now.
virial_frame.parquet exists. Loading now.

Splitting data into train, val, and test sets...

Creating Rips Frame for 1-Homology...
rips_1_frame.parquet exists. Loading now.

Getting train data...



In [5]:
from torch.utils.data import DataLoader
from tqdm import tqdm

In [8]:
val_loader = DataLoader(datset, batch_size = 200, shuffle=True)
for sample in tqdm(val_loader):
    continue

100%|██████████| 540/540 [00:06<00:00, 85.00it/s]
